In [1]:
import polars as pl
import subprocess

In [2]:
dir = "/home/stef/quest_data/hiec/data/sprhea"

df = pl.read_csv(f"{dir}/v3_folded_pt_ns.csv", separator='\t')
df.head()

Entry,Label,Sequence
str,str,str
"""P0A6W3""","""1123""","""MLVWLAEHLVKYYSGFNVFSYLTFRAIVSL…"
"""P9WMW7""","""1123""","""MRQILIAVAVAVTVSILLTPVLIRLFTKQG…"
"""O66465""","""1123""","""MLYQLALLLKDYWFAFNVLKYITFRSFTAV…"
"""Q8MJ30""","""3097;7946""","""MAAAAAGEARRVLVYGGRGALGSRCVQAFR…"
"""P38489""","""3097;7946""","""MDIISVALKRHSTKAFDASKKLTPEQAEQI…"


In [3]:
upids = df["Entry"].to_list()
upids[:5]

['P0A6W3', 'P9WMW7', 'O66465', 'Q8MJ30', 'P38489']

In [4]:
file_paths = [f"gs://public-datasets-deepmind-alphafold-v4/AF-{u}-F1-model_v4.cif" for u in upids]
output_file = 'uniprot_cif_paths.txt' 
with open(output_file, 'w') as file:
    file.write('\n'.join(file_paths))

In [20]:
retrieved_af2 = []
retrieved_path = "/home/stef/quest_data/hiec/data/sprhea/af2_embeds.txt"
with open(retrieved_path, 'r') as file:
    for line in file:
        retrieved_af2.append(line.strip())

print(f"Retrieved {len(retrieved_af2)} AF2 paths")
retrieved_af2[:5]

Retrieved 23940 AF2 paths


['AF-A0A009IHW8-F1-model_v4.cif',
 'AF-A0A017SPL2-F1-model_v4.cif',
 'AF-A0A023GS28-F1-model_v4.cif',
 'AF-A0A023W421-F1-model_v4.cif',
 'AF-A0A023YYV9-F1-model_v4.cif']

In [21]:
tmp = []
for elt in retrieved_af2:
    split_elt = elt.split('-')
    try:
        tmp.append(split_elt[1])
    except:
        print(f"Could not split {elt}")

Could not split af2_embeds.txt


In [22]:
retrieved_af2 = set(tmp)
full_upids = set(upids)
blacklist = full_upids - retrieved_af2
print(f"Found {len(blacklist)} missing AF2 embeddings")

Found 584 missing AF2 embeddings


In [23]:
with open("/home/stef/quest_data/hiec/data/sprhea/af2_blacklist.txt", 'w') as file:
    for item in blacklist:
        file.write(f"{item}\n")